In [37]:
import itertools
import random
import pandas as pd

def initialize_data(players):
    """ 선수 목록 및 초기 데이터 설정 """
    player_games = {player: 0 for player in players}
    player_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    opponent_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    return player_games, player_pairs, opponent_pairs

def can_play_together(team, player_pairs):
    """ 같은 팀에서 1번만 만날 수 있도록 검사 """
    return all(player_pairs[frozenset([p1, p2])] < 1 for p1, p2 in itertools.combinations(team, 2))

def can_play_against(team1, team2, opponent_pairs):
    """ 상대 팀으로 2번까지 만날 수 있도록 검사 """
    return all(opponent_pairs[frozenset([p1, p2])] < 2 for p1 in team1 for p2 in team2)

def add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs):
    """ 매치를 추가하고 기록 업데이트 """
    matches.append((team1, team2))
    for player in team1 + team2:
        player_games[player] += 1
    for p1, p2 in itertools.combinations(team1, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1, p2 in itertools.combinations(team2, 2):
        player_pairs[frozenset([p1, p2])] += 1
    for p1 in team1:
        for p2 in team2:
            opponent_pairs[frozenset([p1, p2])] += 1

def generate_schedule(players):
    """ 경기 스케줄 생성 """
    
    attempt_count = 0
    while True:
        attempt_count += 1
        matches = []
        player_games, player_pairs, opponent_pairs = initialize_data(players)
        possible_combinations = list(itertools.combinations(players, 4))
        random.shuffle(possible_combinations)
        last_match_players = []
        
        for combination in possible_combinations:
            team1, team2 = combination[:2], combination[2:]
            if (all(player_games[player] < 4 for player in combination) and 
                can_play_together(team1, player_pairs) and 
                can_play_together(team2, player_pairs) and 
                can_play_against(team1, team2, opponent_pairs) and
                not any(player in last_match_players for player in combination)):
                
                add_match(matches, team1, team2, player_games, player_pairs, opponent_pairs)
                last_match_players = list(combination)
                
            if all(games == 4 for games in player_games.values()):
                break
        
        if len(matches) == len(players):
            return matches, player_games, attempt_count

def print_results(matches, player_games, players, attempt_count, group_name):
    """ 경기 결과 및 통계 출력 """
    print(f"\n=== {group_name} 경기 결과 ===")
    for i, match in enumerate(matches):
        team1, team2 = match
        print(f"Match {i + 1}: {', '.join(team1)} vs {', '.join(team2)}")
    print("\n각 선수의 경기 횟수:")
    for player, games in player_games.items():
        print(f"{player}: {games}경기")
    match_count_table = pd.DataFrame(0, index=players, columns=players)
    for team1, team2 in matches:
        for p1 in team1:
            for p2 in team2:
                match_count_table.at[p1, p2] += 1
                match_count_table.at[p2, p1] += 1
    print("\n경기에서 만난 횟수 테이블:")
    print(match_count_table)
    print(f"\n경기 생성에 성공한 시도 횟수: {attempt_count}")

# 선수 그룹 정의
player_groups = {
    "남자 그룹 1": ["동환", "매버릭", "밤송이", "비케이", "성현", "소니", "재근", "정현", "주방", "큐", "태보", "Bdup", "프리마", "토니"],
    "남자 그룹 2": ["로센", "명자", "매머드", "미키찬", "브라운", "안씨", "영훈", "전", "지호", "페더러", "푸마", "현민", "Ace", "태규", "DY"],
    "남자 그룹 3": ["김군", "로저", "마크", "성일월드", "스테판", "우디", "이언짱", "제이슨", "쥬니혀니", "쿨쿨", "터보", "황더러", "SJ", "TG"],
    "여자 그룹 1": ["가린", "기쁨", "리디아", "꾸꾸", "다미", "레일라", "미아", "밀리", "유하", "럽테닛", "커피믹스", "Hailey", "연주"],
    "여자 그룹 2": ["젤로", "민크", "뽐", "사리", "수임이", "강진", "이피", "제인", "꼬맹", "제시", "지니", "레이첼", "파란하늘"],
    "여자 그룹 3": ["김지현", "룰루", "문문", "반이", "야금", "우당탕", "크림", "클레어", "Rent", "린다", "아롬"]
}

# random seed
random.seed(40)

# 각 그룹에 대해 경기 생성 및 결과 출력
for group_name, players in player_groups.items():
    matches, player_games, attempt_count = generate_schedule(players)
    print_results(matches, player_games, players, attempt_count, group_name)



=== 남자 그룹 1 경기 결과 ===
Match 1: 성현, 소니 vs 재근, 큐
Match 2: 매버릭, 비케이 vs Bdup, 프리마
Match 3: 밤송이, 소니 vs 큐, 토니
Match 4: 동환, 매버릭 vs 비케이, 태보
Match 5: 재근, 정현 vs 주방, 프리마
Match 6: 매버릭, 밤송이 vs 태보, Bdup
Match 7: 주방, 큐 vs 프리마, 토니
Match 8: 동환, 밤송이 vs 성현, 정현
Match 9: 비케이, 소니 vs 주방, 태보
Match 10: 동환, 재근 vs 큐, Bdup
Match 11: 매버릭, 소니 vs 정현, 토니
Match 12: 성현, 재근 vs 주방, Bdup
Match 13: 밤송이, 비케이 vs 정현, 프리마
Match 14: 동환, 성현 vs 태보, 토니

각 선수의 경기 횟수:
동환: 4경기
매버릭: 4경기
밤송이: 4경기
비케이: 4경기
성현: 4경기
소니: 4경기
재근: 4경기
정현: 4경기
주방: 4경기
큐: 4경기
태보: 4경기
Bdup: 4경기
프리마: 4경기
토니: 4경기

경기에서 만난 횟수 테이블:
      동환  매버릭  밤송이  비케이  성현  소니  재근  정현  주방  큐  태보  Bdup  프리마  토니
동환     0    0    0    1   1   0   0   1   0  1   2     1    0   1
매버릭    0    0    0    1   0   0   0   1   0  0   2     2    1   1
밤송이    0    0    0    0   1   0   0   2   0  1   1     1    1   1
비케이    1    1    0    0   0   0   0   1   1  0   1     1    2   0
성현     1    0    1    0   0   0   1   0   1  1   1     1    0   1
소니     0    0    0    0   0   0   1   1   1 

In [69]:
import pandas as pd

# 각 그룹의 경기 결과 저장
group_results = {}
for group_name, players in player_groups.items():
    matches, player_games, attempt_count = generate_schedule(players)
    group_results[group_name] = matches[:14]  # 최대 14경기 저장

# 경기 일정 테이블 생성
num_rounds = 14
columns = ["시간", "A코트 (여자 그룹 3)", "B코트 (여자 그룹 2)", "C코트 (여자 그룹 1)",
           "D코트 (남자 그룹 3)", "E코트 (남자 그룹 2)", "F코트 (남자 그룹 1)", "G코트"]
schedule_df = pd.DataFrame(columns=columns)
schedule_df["시간"] = range(1, num_rounds + 1)

group_order = ["여자 그룹 3", "여자 그룹 2", "여자 그룹 1", "남자 그룹 3", "남자 그룹 2", "남자 그룹 1"]

i = 0
for round_num in range(num_rounds):
    for col, group_name in zip(columns[1:], group_order):
        if group_name in group_results and i < len(group_results[group_name]):
            match = group_results[group_name][i]
            team1, team2 = match[0], match[1]
            schedule_df.at[round_num, col] = f"{', '.join(team1)} vs {', '.join(team2)}"
    i += 1  # 순서대로 그룹마다 경기 추가

# 빈 값이 있는 경우 "경기 없음"으로 채우기
schedule_df.fillna("경기 없음", inplace=True)

# 결과 출력
print(schedule_df)


    시간        A코트 (여자 그룹 3)        B코트 (여자 그룹 2)           C코트 (여자 그룹 1)  \
0    1    우당탕, 크림 vs 린다, 아롬  젤로, 제시 vs 레이첼, 파란하늘    리디아, 레일라 vs 밀리, 커피믹스   
1    2   김지현, 룰루 vs 문문, 클레어      뽐, 강진 vs 꼬맹, 지니   다미, 미아 vs 럽테닛, Hailey   
2    3   반이, 크림 vs Rent, 린다   민크, 이피 vs 제시, 파란하늘       가린, 기쁨 vs 레일라, 연주   
3    4   김지현, 문문 vs 야금, 클레어    사리, 수임이 vs 강진, 꼬맹       리디아, 꾸꾸 vs 미아, 유하   
4    5  룰루, 우당탕 vs 크림, Rent   젤로, 이피 vs 지니, 파란하늘     다미, 럽테닛 vs 커피믹스, 연주   
5    6    문문, 반이 vs 클레어, 아롬    뽐, 수임이 vs 꼬맹, 레이첼   가린, 레일라 vs 미아, Hailey   
6    7    룰루, 야금 vs 우당탕, 린다     젤로, 사리 vs 제인, 지니        기쁨, 다미 vs 유하, 연주   
7    8    김지현, 반이 vs 크림, 아롬     민크, 뽐 vs 강진, 레이첼       가린, 꾸꾸 vs 밀리, 럽테닛   
8    9  문문, 야금 vs 우당탕, Rent     사리, 제인 vs 꼬맹, 제시  기쁨, 미아 vs 커피믹스, Hailey   
9   10    룰루, 반이 vs 클레어, 린다   민크, 수임이 vs 이피, 레이첼      리디아, 유하 vs 럽테닛, 연주   
10  11  김지현, 야금 vs Rent, 아롬      뽐, 제인 vs 제시, 지니    기쁨, 꾸꾸 vs 밀리, Hailey   
11  12                경기 없음    젤로, 민크 vs 수임이, 이피     다미, 레일라 vs 유하, 커피믹스   
12  13      

In [70]:
import pandas as pd

# 각 그룹의 경기 결과 저장
group_results = {}
for group_name, players in player_groups.items():
    matches, player_games, attempt_count = generate_schedule(players)
    group_results[group_name] = matches[:14]  # 최대 14경기 저장

# 경기 일정 테이블 생성
num_rounds = 14
columns = ["시간"]
group_order = ["여자 그룹 3", "여자 그룹 2", "여자 그룹 1", "남자 그룹 3", "남자 그룹 2", "남자 그룹 1"]

# 각 그룹별 4개 칼럼 생성
for group_name in group_order:
    columns.extend([f"{group_name}_1", f"{group_name}_2", f"{group_name}_3", f"{group_name}_4"])

schedule_df = pd.DataFrame(columns=columns)
schedule_df["시간"] = range(1, num_rounds + 1)

i = 0
for round_num in range(num_rounds):
    for group_name in group_order:
        if group_name in group_results and i < len(group_results[group_name]):
            match = group_results[group_name][i]
            team1, team2 = match[0], match[1]
            players_list = team1 + team2
            for j in range(4):
                schedule_df.at[round_num, f"{group_name}_{j+1}"] = players_list[j]  # 각 칼럼에 한 명씩 배정
    i += 1  # 순서대로 그룹마다 경기 추가

# 빈 값이 있는 경우 "경기 없음"으로 채우기
schedule_df.fillna("경기 없음", inplace=True)

# 결과 출력
display(schedule_df)

schedule_df.to_csv("schedule.csv", index=False)

,시간,여자 그룹 3_1,여자 그룹 3_2,여자 그룹 3_3,여자 그룹 3_4,여자 그룹 2_1,여자 그룹 2_2,여자 그룹 2_3,여자 그룹 2_4,여자 그룹 1_1,...,남자 그룹 3_3,남자 그룹 3_4,남자 그룹 2_1,남자 그룹 2_2,남자 그룹 2_3,남자 그룹 2_4,남자 그룹 1_1,남자 그룹 1_2,남자 그룹 1_3,남자 그룹 1_4
0,1,김지현,야금,Rent,아롬,수임이,강진,꼬맹,지니,가린,...,제이슨,쥬니혀니,명자,안씨,페더러,DY,매버릭,소니,프리마,토니
1,2,반이,우당탕,크림,클레어,젤로,사리,레이첼,파란하늘,꾸꾸,...,이언짱,터보,영훈,전,Ace,태규,밤송이,비케이,정현,Bdup
2,3,룰루,문문,린다,아롬,민크,꼬맹,제시,지니,기쁨,...,스테판,TG,명자,미키찬,현민,DY,동환,성현,재근,태보
3,4,야금,우당탕,클레어,Rent,뽐,사리,이피,레이첼,리디아,...,쥬니혀니,쿨쿨,매머드,브라운,영훈,페더러,매버릭,정현,Bdup,토니
4,5,김지현,문문,반이,아롬,민크,제인,꼬맹,파란하늘,기쁨,...,황더러,TG,로센,지호,푸마,Ace,비케이,주방,큐,태보
5,6,룰루,우당탕,크림,린다,젤로,수임이,강진,제시,리디아,...,우디,SJ,명자,브라운,페더러,태규,밤송이,재근,정현,토니
6,7,문문,반이,야금,Rent,뽐,이피,제인,파란하늘,가린,...,쥬니혀니,황더러,로센,영훈,전,Ace,동환,매버릭,소니,주방
7,8,김지현,룰루,우당탕,클레어,민크,수임이,제시,레이첼,기쁨,...,쿨쿨,SJ,매머드,미키찬,안씨,DY,비케이,재근,태보,토니
8,9,문문,크림,Rent,린다,젤로,뽐,지니,파란하늘,리디아,...,우디,제이슨,전,지호,페더러,푸마,매버릭,성현,주방,큐
9,10,룰루,반이,야금,아롬,강진,이피,꼬맹,제시,다미,...,터보,SJ,로센,브라운,안씨,영훈,비케이,소니,Bdup,프리마


In [24]:
# 매칭돌리는 함수
import itertools
import random
import pandas as pd

# 선수 목록

players_m_1 = ["동환", "매버릭", "밤송이", "비케이", "성현", "소니", "재근", "정현", "주방", "큐", "태보", "Bdup", "프리마", "토니"]
players_m_2 = ["로센", "명자", "매머드", "미키찬", "브라운", "안씨", "영훈", "전", "지호", "페더러", "푸마", "현민", "Ace", "태규", "DY"]
players_m_3 = ["김군", "로저", "마크", "성일월드", "스테판", "우디", "이언짱", "제이슨", "쥬니혀니", "쿨쿨", "터보", "황더러", "SJ", "TG"]
players_f_1 = ["가린", "기쁨", "리디아", "꾸꾸", "다미", "레일라", "미아", "밀리", "유하", "럽테닛", "커피믹스", "Hailey", "연주"]
players_f_2 = ["젤로", "민크", "뽐", "사리", "수임이", "강진", "이피", "제인", "꼬맹", "제시", "지니", "레이첼", "파란하늘"]
players_f_3 = ["김지현", "룰루", "문문", "반이", "야금", "우당탕", "크림", "클레어", "Rent", "린다", "아롬"]

players = players_m_3

# 랜덤시드 
random.seed(42)

# 각 선수의 경기 횟수와 함께 경기 기록을 추적
attempt_count = 0

while True:
    attempt_count += 1
    matches = []
    player_games = {player: 0 for player in players}
    player_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}
    opponent_pairs = {frozenset([p1, p2]): 0 for p1, p2 in itertools.combinations(players, 2)}

    # 가능한 모든 조합 생성
    possible_combinations = list(itertools.combinations(players, 4))
    random.shuffle(possible_combinations)

    # 매치 생성 함수
    def can_play_together(team):
        for p1, p2 in itertools.combinations(team, 2):
            if player_pairs[frozenset([p1, p2])] >= 1:  # 같은 팀으로 1번만 만남
                return False
        return True

    def can_play_against(team1, team2):
        for p1 in team1:
            for p2 in team2:
                if opponent_pairs[frozenset([p1, p2])] >= 2:  # 상대편으로는 2번만 만남
                    return False
        return True

    def add_match(team1, team2):
        matches.append((team1, team2))
        for player in team1 + team2:
            player_games[player] += 1
        for p1, p2 in itertools.combinations(team1, 2):
            player_pairs[frozenset([p1, p2])] += 1
        for p1, p2 in itertools.combinations(team2, 2):
            player_pairs[frozenset([p1, p2])] += 1
        for p1 in team1:
            for p2 in team2:
                opponent_pairs[frozenset([p1, p2])] += 1

    # 연속 경기 방지: 이전 경기에 출전한 선수가 다음 경기에는 출전하지 않도록 추적
    last_match_players = []

    # 매치 생성
    for combination in possible_combinations:
        team1 = combination[:2]
        team2 = combination[2:]

        # 각 선수의 경기 횟수가 4 미만이고, 같은 선수가 연속으로 경기에 참여하지 않으며,
        # 같은 선수와 3번 이상 경기를 하지 않았을 때만 경기를 추가
        if (all(player_games[player] < 4 for player in combination) and 
            can_play_together(team1) and 
            can_play_together(team2) and 
            can_play_against(team1, team2) and
            not any(player in last_match_players for player in combination)):

            add_match(team1, team2)

            # 이전 경기에 참여한 선수들 업데이트
            last_match_players = list(combination)

        # 모든 선수의 경기가 완료되었으면 중지
        if all(games == 4 for games in player_games.values()):
            break

    # 경기 수가 14인 경우에만 종료
    if len(matches) == len(players):
        break

# 결과 출력
for i, match in enumerate(matches):
    team1, team2 = match
    print(f"Match {i + 1}: {', '.join(team1)} vs {', '.join(team2)}")

print("\n각 선수의 경기 횟수:")
for player, games in player_games.items():
    print(f"{player}: {games}경기")

# 경기에서 만난 횟수 테이블 생성
match_count_table = pd.DataFrame(0, index=players, columns=players)
for team1, team2 in matches:
    for p1 in team1:
        for p2 in team2:
            match_count_table.at[p1, p2] += 1
            match_count_table.at[p2, p1] += 1

print("\n경기에서 만난 횟수 테이블:")
print(match_count_table)

# 시도 횟수 출력
print(f"\n경기 생성에 성공한 시도 횟수: {attempt_count}")


Match 1: 마크, 제이슨 vs 터보, TG
Match 2: 김군, 쥬니혀니 vs 쿨쿨, SJ
Match 3: 마크, 터보 vs 황더러, TG
Match 4: 로저, 성일월드 vs 스테판, 제이슨
Match 5: 김군, 마크 vs 우디, 쿨쿨
Match 6: 로저, 터보 vs 황더러, SJ
Match 7: 성일월드, 이언짱 vs 쿨쿨, TG
Match 8: 마크, 스테판 vs 우디, 쥬니혀니
Match 9: 로저, 이언짱 vs SJ, TG
Match 10: 김군, 성일월드 vs 쥬니혀니, 황더러
Match 11: 스테판, 우디 vs 이언짱, 쿨쿨
Match 12: 제이슨, 쥬니혀니 vs 터보, 황더러
Match 13: 김군, 로저 vs 성일월드, 스테판
Match 14: 우디, 이언짱 vs 제이슨, SJ

각 선수의 경기 횟수:
김군: 4경기
로저: 4경기
마크: 4경기
성일월드: 4경기
스테판: 4경기
우디: 4경기
이언짱: 4경기
제이슨: 4경기
쥬니혀니: 4경기
쿨쿨: 4경기
터보: 4경기
황더러: 4경기
SJ: 4경기
TG: 4경기

경기에서 만난 횟수 테이블:
      김군  로저  마크  성일월드  스테판  우디  이언짱  제이슨  쥬니혀니  쿨쿨  터보  황더러  SJ  TG
김군     0   0   0     1    1   1    0    0     1   2   0    1   1   0
로저     0   0   0     1    2   0    0    1     0   0   0    1   2   1
마크     0   0   0     0    0   2    0    0     1   1   1    1   0   2
성일월드   1   1   0     0    1   0    0    1     1   1   0    1   0   1
스테판    1   2   0     1    0   1    1    0     1   1   0    0   0   0
우디     1   0   2     0    1   0   